# Ansys CPFE UMAT Example

In [5]:
import sys
import os

module_path = os.path.abspath(os.path.join('..','..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from pathlib import Path

# Result directory
result_dir = Path("Results/UMAT")
# Create the directory if it doesn't exist
result_dir.mkdir(parents=True, exist_ok=True)

In [6]:
# Material parameters
material = 'Titanium'
elasticType = 'HCP'
ElasticConstants = [154000, 86000, 183000, 67300, 46700] # C11, C12, C13, C33, C44 in MPa
ElasticModulus = 120000  # MPa
ShearModulus = 45000  # MPa
PoissonRatio = 0.34
CTE = 1e-05  # Coefficient of thermal expansion in 1/K
EUL1 = 159  # Euler angle 1 in degrees
EUL2 = 65  # Euler angle 2 in degrees
EUL3 = 60  # Euler angle 3 in degrees
nSlipFamilies = 5 # Number of slip families
InitialHardness_per_slipFamily = [12,8,80,24,80] # MPa
HardnessModulus_per_slipFamily = [18, 18,180,48,180] # MPa
SaturationHardness_per_slipFamily = [12,12,12,12,12] # MPa

# material = 'Copper'
# elasticType = 'CUBIC'
# ElasticConstants = [169000, 121000, 77100] # C11, C12, C44 in MPa
# ElasticModulus = 110000  # MPa
# ShearModulus = 45000  # MPa
# PoissonRatio = 0.34
# CTE = 1e-05  # Coefficient of thermal expansion in 1/K
# EUL1 = 90  # Euler angle 1 in degrees
# EUL2 = 45  # Euler angle 2 in degrees
# EUL3 = 0  # Euler angle 3 in degrees
# nSlipFamilies = 1 # Number of slip families
# InitialHardness_per_slipFamily = [80] # MPa
# HardnessModulus_per_slipFamily = [180] # MPa
# SaturationHardness_per_slipFamily = [12] # MPa


In [7]:
# Other parameters
Tref = 298.0 # Reference temperature in K
NamedSelection = 'SX'

In [8]:
Header = """! Command snippet created by ANSYS Workbench

!   Commands inserted into this file will be executed just prior to the ANSYS SOLVE command.
!   These commands may supersede command settings set by Workbench.

!   Active UNIT system in Workbench when this object was created:  Metric (um, kg, uN, s, V, mA)
!   NOTE:  Any data that requires units (such as mass) is assumed to be in the consistent solver unit system.
!                See Solving Units in the help system for more information.
/prep7
! Begin writing variables"""



if elasticType == 'CUBIC':
    assert(len(ElasticConstants) == 3)
    C11, C12, C44 = ElasticConstants
    E_ = (C11**2 + C12*C11 - 2*C12**2) / (C11 + C12)
    v_ = C12 / (C11 + C12)
    G_ = C44
    ih = InitialHardness_per_slipFamily[0]
    hm = HardnessModulus_per_slipFamily[0]
    sh = SaturationHardness_per_slipFamily[0]
elif elasticType == 'HCP':
    C11, C12, C13, C33, C44 = ElasticConstants
    E_ = ElasticModulus
    v_ = PoissonRatio
    G_ = C44
    ih1, ih2, ih3, ih4, ih5 = InitialHardness_per_slipFamily
    hm1, hm2, hm3, hm4, hm5 = HardnessModulus_per_slipFamily
    sh1, sh2, sh3, sh4, sh5 = SaturationHardness_per_slipFamily

UMAT = f"""
! Begin writing grain materials
*SET,matid,1
TB,ELAS,matid,,9,
TBDATA,1,{E_},{E_},{E_}, {G_},{G_},{G_}, {v_},
TBDATA,7,{v_}, {v_}, {v_},
TB,CTE,matid,
TBDATA,1,{CTE},{CTE},{CTE}
TB,PLAS,matid
TB,XTAL,matid,,3,ORIE
TBDATA,1,{EUL1},{EUL2},{EUL3}
"""
if elasticType == 'CUBIC':
    UMAT += f"""
TB,XTAL,matid,,1,NSLFAM
TBDATA,1,{nSlipFamilies}
TB,XTAL,matid,,1,FORM
TBDATA,1,2
TB,XTAL,matid,,6,XPARAM
TBDATA,1,1,0,3,12,0,1
TB,XTAL,matid,1,6,HARD
TBTEMP, {Tref},
TBDATA,1,{ih},{hm},{sh},.0,2,0,1.4
TB,XTAL,matid,1,6,FLFCC
TBTEMP, {Tref},
TBDATA,1,1732000.0,0.131,1.1,0.7,2.5e-19,1
"""
elif elasticType == 'HCP':
    UMAT += f"""
TB,XTAL,matid,,1,NSLFAM
TBDATA,1,{nSlipFamilies}
TB,XTAL,matid,,1,FORM
TBDATA,1,1
TB,XTAL,matid,,10,XPARAM
TBDATA,1,2,0,45,30,1,1,
TBDATA,7,1,1,1,1
TB,XTAL,matid,1,18,HARD
TBTEMP, {Tref},
TBDATA,1,{ih1},{ih2},{ih3},{ih4},{ih5},
TBDATA,7,{hm1},{hm2},{hm3},{hm4},{hm5},
TBDATA,13,{sh1},{sh2},{sh3},{sh4},{sh5},.0,2,0,1.4
TB,XTAL,matid,1,3,FLHCP
TBTEMP, {Tref},
TBDATA,1,0.001,6.25,1.587
"""
UMAT += f"""
CMSEL,S,{NamedSelection}
EMODIF,ALL,MAT,matid
ALLSEL,ALL

allsel,all
TREF, {Tref}
TUNIF, {Tref}
/solu
"""

print(UMAT)

#Save to file
title = f"UMAT_{material}.txt"
with open(result_dir / title, "w") as file:
    file.write(Header)
    file.write(UMAT)


! Begin writing grain materials
*SET,matid,1
TB,ELAS,matid,,9,
TBDATA,1,120000,120000,120000, 46700,46700,46700, 0.34,
TBDATA,7,0.34, 0.34, 0.34,
TB,CTE,matid,
TBDATA,1,1e-05,1e-05,1e-05
TB,PLAS,matid
TB,XTAL,matid,,3,ORIE
TBDATA,1,159,65,60

TB,XTAL,matid,,1,NSLFAM
TBDATA,1,5
TB,XTAL,matid,,1,FORM
TBDATA,1,1
TB,XTAL,matid,,10,XPARAM
TBDATA,1,2,0,45,30,1,1,
TBDATA,7,1,1,1,1
TB,XTAL,matid,1,18,HARD
TBTEMP, 298.0,
TBDATA,1,12,8,80,24,80,
TBDATA,7,18,18,180,48,180,
TBDATA,13,12,12,12,12,12,.0,2,0,1.4
TB,XTAL,matid,1,3,FLHCP
TBTEMP, 298.0,
TBDATA,1,0.001,6.25,1.587

CMSEL,S,SX
EMODIF,ALL,MAT,matid
ALLSEL,ALL

allsel,all
TREF, 298.0
TUNIF, 298.0
/solu

